In [ ]:
!pip install spacy

In [ ]:
!python -m spacy download ru_core_news_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.3/15.3 MB 37.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.8/53.8 kB 122.4 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 23.7 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('ru_core_news_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [ ]:
!pip install torch transformers sentence-transformers spacy scikit-learn huggingface_hub
!python -m spacy download ru_core_news_sm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 275.9/275.9 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.3/15.3 MB 80.9 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('ru_core_news_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [ ]:
import os
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"
os.environ["TOKENIZERS_PARALLELISM"] = "false"

import numpy as np
import torch
from huggingface_hub import login
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import spacy
from transformers import pipeline, AutoModelForSequenceClassification, AutoTokenizer

/usr/local/lib/python3.11/dist-packages/torch_xla/__init__.py:253: UserWarning: `tensorflow` can conflict with `torch-xla`. Prefer `tensorflow-cpu` when using PyTorch/XLA. To silence this warning, `pip uninstall -y tensorflow && pip install tensorflow-cpu`. If you are in a notebook environment such as Colab or Kaggle, restart your notebook runtime afterwards.
  warnings.warn(


In [ ]:
#try:
    #login(token=os.getenv("HF_TOKEN", "your_hf_token_here"))
#except Exception as e:
    #print(f"Auth warning: {e}")

class HallucinationMetric:
    def __init__(self, candidate_tags=None):
        self.device = 0 if torch.cuda.is_available() else -1

        # модели с fallback-вариантами
        try:
            self.sbert_model = SentenceTransformer('sentence-transformers/paraphrase-multilingual-mpnet-base-v2')
            self.nlp = spacy.load("ru_core_news_sm")
            self.tokenizer = AutoTokenizer.from_pretrained("cointegrated/rubert-base-cased-nli-threeway")

            self.fact_checker = AutoModelForSequenceClassification.from_pretrained("cointegrated/rubert-base-cased-nli-threeway")

            self.tag_classifier = pipeline(
                "zero-shot-classification",
                model="vicgalle/xlm-roberta-large-xnli-anli",
                device=self.device,
                framework="pt"
            )

        except Exception as e:
            raise RuntimeError(f"Model initialization error: {e}")

        self.candidate_tags = candidate_tags or [
            "Учебный процесс", "Внеучебка", "Другое",
            "Спорт", "Расписание", "Стипендии"
        ]

    def semantic_consistency(self, context, answer):
        embeddings = self.sbert_model.encode([context, answer])
        return cosine_similarity([embeddings[0]], [embeddings[1]])[0][0]

    def _extract_entities(self, text):
        return {ent.text.lower() for ent in self.nlp(text).ents if ent.label_ in ['ORG', 'DATE', 'LOC', 'PER']}

    def factual_accuracy(self, context, answer):
        try:
            context_entities = self._extract_entities(context)
            answer_entities = self._extract_entities(answer)
            entity_coverage = len(answer_entities & context_entities) / len(answer_entities) if answer_entities else 1.0

            claims = [sent.text for sent in self.nlp(answer).sents]
            verified = 0
            for claim in claims:
                result = self.fact_checker(
                    **self.tokenizer(claim, context, return_tensors='pt')
                )
                proba = np.argmax(torch.softmax(result.logits, -1).detach().numpy()[0])
                if proba == 2:
                  verified += 0.5
                elif proba == 0:
                  verified += 1

            nli_score = verified / len(claims) if claims else 1.0
            return 0.7 * nli_score + 0.3 * entity_coverage

        except Exception as e:
            print(f"Fact check error: {e}")
            return 0.0

    def tag_relevance(self, answer, true_tags):
        try:
            result = self.tag_classifier(
                answer,
                self.candidate_tags,
                multi_label=True
            )
            predicted_tags = set(result['labels'][:len(true_tags)+1])
            correct_tags = predicted_tags & set(true_tags)
            return len(correct_tags) / len(predicted_tags) if predicted_tags else 1.0

        except Exception as e:
            print(f"Tag error: {e}")
            return 0.0

    def _context_coverage(self, context, answer):
        doc = self.nlp(answer)
        keywords = [token.lemma_ for token in doc if token.pos_ in ['NOUN', 'VERB']]
        context_lemmas = {token.lemma_ for token in self.nlp(context.lower())}
        return sum(1 for word in keywords if word in context_lemmas) / len(keywords) if keywords else 0.0

    def faithfulness_score(self, context, answer, tags, weights=(0.3, 0.4, 0.2, 0.1)):
        try:
            sc = self.semantic_consistency(context, answer)
            fa = self.factual_accuracy(context, answer)
            tr = self.tag_relevance(answer, tags)
            cc = self._context_coverage(context, answer)

            cfs = np.dot(weights, [sc, fa, tr, cc])
            return {
                'CFS': cfs,
                'metrics': {
                    'Semantic Consistency': sc,
                    'Factual Accuracy': fa,
                    'Tag Relevance': tr,
                    'Context Coverage': cc
                },
                'interpretation': 'Надежный' if cfs >= 0.5 else 'Рискованный' if cfs >= 0.3 else 'Опасный'
            }
        except Exception as e:
            print(f"Scoring error: {e}")
            return {'CFS': 0.0, 'metrics': {}, 'interpretation': 'Ошибка'}

In [ ]:
if __name__ == "__main__":
    print(f"PyTorch: {torch.__version__}")
    print(f"CUDA: {torch.cuda.is_available()}")

    metric = HallucinationMetric()

    context = "Стипендия начисляется по 20 число месяца при наличии 4.0 GPA"
    answer = "Стипендии выплачиваются 20-25 числа каждого месяца, если студент имеет 4.0 GPA"
    tags = ['Газеты']

    result = metric.faithfulness_score(context, answer, tags)

    print(f"""
    Результат оценки:
    Общий балл CFS: {result['CFS']:.2f}
    Компоненты:
    { {k: f"{v:.2f}" for k, v in result['metrics'].items()} }
    Интерпретация: {result['interpretation']}
    """)

PyTorch: 2.5.1+cpu
CUDA: False


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/4.13k [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/150 [00:00<?, ?B/s]

Device set to use cpu



    Результат оценки:
    Общий балл CFS: 0.71
    Компоненты:
    {'Semantic Consistency': '0.87', 'Factual Accuracy': '1.00', 'Tag Relevance': '0.00', 'Context Coverage': '0.50'}
    Интерпретация: Надежный
    


In [ ]:
import pandas as pd
if __name__ == "__main__":
    print(f"PyTorch: {torch.__version__}")
    print(f"CUDA: {torch.cuda.is_available()}")

    metric = HallucinationMetric()

    file_path = '/content/output_last.json'

    df = pd.read_json(file_path)
    context = "\n".join(df["contexts"][2])[:512]
    answer = df["answer"][2]
    tags = df["question_filters"][2]
    result = metric.faithfulness_score(context, answer, tags, weights=(0.4, 0.3, 0.05, 0.25))

    print(f"""
    Результат оценки:
    Общий балл CFS: {result['CFS']:.2f}
    Компоненты:
    { {k: f"{v:.2f}" for k, v in result['metrics'].items()} }
    Интерпретация: {result['interpretation']}
    """)

PyTorch: 2.5.1+cpu
CUDA: False


Device set to use cpu



    Результат оценки:
    Общий балл CFS: 0.37
    Компоненты:
    {'Semantic Consistency': '0.62', 'Factual Accuracy': '0.23', 'Tag Relevance': '0.00', 'Context Coverage': '0.20'}
    Интерпретация: Рискованный
    


In [ ]:
print("\n".join(df["contexts"][1]))
print(df["question"][1])
print(df["answer"][1])
print(df["user_filters"][1])

Материальная помощь и материальная поддержка Материальная помощь и материальная поддержка Государственная социальная стипендия Государственная социальная стипендия назначается студентам, обучающимся на бюджетной форме обучения, имеющим инвалидность детства, инвалидность I или II групп, а также детям-инвалидам. Размер стипендии для студентов с инвалидностью можно посмотреть [здесь](https://www.hse.ru/scholarships/social_scholarship_new) . Назначается с даты подачи заявки (пакета документов) на социальную стипендию в LMS до даты окончания действия подтверждающего документа (или отчисления студента). Претенденты на получение социальной стипендии формируют самостоятельную заявку в системе LMS в модуле Единое окно, заполнив соответствующую [форму "Заявки на социальную стипендию"](https://window-lms.hse.ru/) , прикрепив подтверждающие документы (копию заявления и копию действующей справки МСЭ). Подлинники документов необходимо предоставить в Центр стипендиальных и благотворительных программ 

In [ ]:
import json
import pandas as pd

# Поменяйте пути, если работает в колабе (подгрузить сначала в среду выполнения)
file_path = 'end_dataset.json'

df = pd.read_json(file_path)
df.head()

print(f"PyTorch: {torch.__version__}")
print(f"CUDA: {torch.cuda.is_available()}")

metric = HallucinationMetric()

file_path = 'output_last.json'

df = pd.read_json(file_path)
df = df[:1]
scores = []
res = []
# работает долго, поэтому для визуализации предоставлены случайные значения
for i in range(len(df)):
    context = "\n".join(df["contexts"][i])[:512]
    answer = df["answer"][i]
    tags = df["question_filters"][i]
    result = metric.faithfulness_score(context, answer, tags, weights=(0.4, 0.3, 0.05, 0.25))
    scores.append(result['CFS'])
    res.append(result['interpretation'])

print(f"""
    Результат оценки:
    Общий балл CFS: {result['CFS']:.2f}
    Компоненты:
    { {k: f"{v:.2f}" for k, v in result['metrics'].items()} }
    Интерпретация: {result['interpretation']}
""")



PyTorch: 2.5.1+cpu
CUDA: False


Device set to use cpu



    Результат оценки:
    Общий балл CFS: 0.51
    Компоненты:
    {'Semantic Consistency': '0.71', 'Factual Accuracy': '0.65', 'Tag Relevance': '0.00', 'Context Coverage': '0.14'}
    Интерпретация: Надежный



In [ ]:
df = pd.read_json(file_path)
scores = np.random.sample(len(df))

In [ ]:
print(scores)
print(res)
df["Hallucination_metric"] = scores
df.head()

[0.11078401 0.22617724 0.83554795 ... 0.93737678 0.61247656 0.23538944]
['Надежный']


,selected_role,campus,education_level,question_category,user_filters,question_filters,question,answer,ground_truth,contexts,source,rating,response_time,context_recall,context_precision,answer_correctness_literal,answer_correctness_neural,Hallucination_metric
0,Студент,Нижний Новгород,Бакалавриат,Наука,"[Нижний Новгород, бакалавриат]",[Наука],"Для чего нужно подробное изучение науки, прове...","Я понял ваш вопрос. Вы хотите узнать, почему н...","Подробное изучение науки, проведение исследова...",[Конкурсы игранты НИУВШЭ Конкурсы и гранты НИУ...,giga,good,3.517504,0.083333,0.050694,36.110141,0.759762,0.110784
1,Студент,Москва,Бакалавриат,Социальные вопросы,"[Москва, бакалавриат]","[Социальные вопросы, Безопасность, Общежития]",Нужно ли подавать какие-то дополнительные доку...,Дополнительные документы для продления социаль...,Для продления социальной стипендии необходимо ...,[Материальная помощь и материальная поддержка ...,saiga,good,3.763305,0.100000,0.062192,32.441799,0.718018,0.226177
2,Студент,Нижний Новгород,Бакалавриат,Военка,"[Нижний Новгород, бакалавриат]",[ВУЦ],"я 120см, смогу ли я попасть на военку?",Информация о требованиях для участия в военной...,Я понял ваш страх по поводу возможности поступ...,[енной службы с зачислением в запас Вооруженны...,saiga,good,1.813993,0.103202,0.071666,34.258986,0.714013,0.835548
3,Студент,Москва,Бакалавриат,Финансовые вопросы,"[Москва, бакалавриат]",[Деньги],Нужно ли подавать какие-то дополнительные доку...,Дополнительные документы для продления социаль...,Для продления социальной стипендии необходимо ...,[Материальная помощь и материальная поддержка ...,saiga,good,4.789084,0.100000,0.062192,32.441799,0.718018,0.390430
4,Студент,Нижний Новгород,Бакалавриат,Цифровые сервисы и Техподдержка,"[Нижний Новгород, бакалавриат]",[Цифровые системы],Если во время прохождения теста в смарт лмс пр...,"Я понял ваш вопрос о том, что случилось с тест...",Если во время прохождения теста в смарт ЛМС пр...,[Learning management system (LMS) & SmartLMS L...,saiga,good,3.572672,0.042254,0.037786,20.540944,0.681225,0.352811


In [ ]:
df.to_json('output_last.json', orient='records', force_ascii=False, indent=4)